# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using GraphLab Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up graphlab create

In [1]:
import graphlab

In [2]:
import numpy as np

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [3]:
sales = graphlab.SFrame('kc_house_data.gl/')

[INFO] 1452080438 : INFO:     (initialize_globals_from_environment:282): Setting configuration variable GRAPHLAB_FILEIO_ALTERNATIVE_SSL_CERT_FILE to C:\Users\margrami\AppData\Local\Dato\Dato Launcher\lib\site-packages\certifi\cacert.pem
1452080438 : INFO:     (initialize_globals_from_environment:282): Setting configuration variable GRAPHLAB_FILEIO_ALTERNATIVE_SSL_CERT_DIR to 
This non-commercial license of GraphLab Create is assigned to margrami221@yahoo.com and will expire on October 13, 2016. For commercial licensing options, visit https://dato.com/buy/.

[INFO] Start server at: ipc:///tmp/graphlab_server-6332 - Server binary: C:\Users\margrami\AppData\Local\Dato\Dato Launcher\lib\site-packages\graphlab\unity_server.exe - Server log: C:\Users\margrami\AppData\Local\Temp\graphlab_server_1452080438.log.0
[INFO] GraphLab Server Version: 1.7.1


# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [4]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [5]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in GraphLab Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [6]:
model_all = graphlab.linear_regression.create(sales, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=1e10)

PROGRESS: Linear regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 21613
PROGRESS: Number of features          : 17
PROGRESS: Number of unpacked features : 17
PROGRESS: Number of coefficients    : 18
PROGRESS: Starting Accelerated Gradient (FISTA)
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+-----------+--------------+--------------------+---------------+
PROGRESS: | Iteration | Passes   | Step size | Elapsed Time | Training-max_error | Training-rmse |
PROGRESS: +-----------+----------+-----------+--------------+--------------------+---------------+
PROGRESS: Tuning step size. First iteration could take longer than subsequent iterations.
PROGRESS: | 1         | 2        | 0.000002  | 1.203069     | 6962915.603493     | 426631.749026 |
PROGRESS: | 2         | 3        | 0.000002  | 1.221070     | 6843144.200219     | 392488.929838 |
PROGRESS: | 3         | 4      

Find what features had non-zero weight.

In [7]:
model_all.coefficients.print_rows(18, 3)


+------------------+-------+---------------+
|       name       | index |     value     |
+------------------+-------+---------------+
|   (intercept)    |  None |  274873.05595 |
|     bedrooms     |  None |      0.0      |
| bedrooms_square  |  None |      0.0      |
|    bathrooms     |  None | 8468.53108691 |
|   sqft_living    |  None | 24.4207209824 |
| sqft_living_sqrt |  None | 350.060553386 |
|     sqft_lot     |  None |      0.0      |
|  sqft_lot_sqrt   |  None |      0.0      |
|      floors      |  None |      0.0      |
|  floors_square   |  None |      0.0      |
|    waterfront    |  None |      0.0      |
|       view       |  None |      0.0      |
|    condition     |  None |      0.0      |
|      grade       |  None | 842.068034898 |
|    sqft_above    |  None | 20.0247224171 |
|  sqft_basement   |  None |      0.0      |
|     yr_built     |  None |      0.0      |
|   yr_renovated   |  None |      0.0      |
+------------------+-------+---------------+
[18 rows x

Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [8]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [ ]:
def polynomial_lasso_regression(data, deg, l1_penalty):
    model = graphlab.linear_regression.create(polynomial_features(data,deg), 
                                              target='Y', l2_penalty=0.,
                                              l1_penalty=l1_penalty,
                                              validation_set=None, 
                                              solver='fista', verbose=False,
                                              max_iterations=3000, convergence_threshold=1e-10)
    return model

In [9]:
L1_array = np.logspace(1, 7, num=13)

In [38]:
L1_array = np.logspace(1, 7, num=13)
RSS = []


for l1_penalty in L1_array:
    model_all = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=l1_penalty, verbose = False)
    RSS.append(((validation['price'] - model_all.predict(validation))**2).sum())


In [40]:
selected_l1_penalty  = L1_array[RSS.index(min(RSS))] 
print 'l1 seleccionada:   '+ str(selected_l1_penalty)
print 'valor min de RSS:  '+ str(min(RSS))

l1 seleccionada:   10.0
valor min de RSS:  6.25766285142e+14


In [41]:
model_selected = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=10, verbose = False)

In [42]:
 print 'number of nonzeros = %d' % (model_selected.coefficients['value']).nnz()

number of nonzeros = 18


In [43]:
model_all.coefficients.print_rows(18, 3)

+------------------+-------+--------------------+
|       name       | index |       value        |
+------------------+-------+--------------------+
|   (intercept)    |  None |   19604.7163508    |
|     bedrooms     |  None |   7922.62023075    |
| bedrooms_square  |  None |   928.112147889    |
|    bathrooms     |  None |    25392.743443    |
|   sqft_living    |  None |   39.1083048767    |
| sqft_living_sqrt |  None |   1123.95233925    |
|     sqft_lot     |  None | -0.000823987151992 |
|  sqft_lot_sqrt   |  None |   147.859054391    |
|      floors      |  None |    21154.353574    |
|  floors_square   |  None |    12871.042877    |
|    waterfront    |  None |   598538.378522    |
|       view       |  None |   93089.3048849    |
|    condition     |  None |   6584.64358167    |
|      grade       |  None |   6200.37319739    |
|    sqft_above    |  None |   43.2627983054    |
|  sqft_basement   |  None |   122.027231745    |
|     yr_built     |  None |   9.39509533202    |


*** QUIZ QUESTIONS ***
1. What was the best value for the `l1_penalty`? 10
2. What is the RSS on TEST data of the model with the best `l1_penalty`? valor min de RSS:  6.30973667812e+14

***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have? : 18

# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [ ]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [31]:
l1_penalty_values = np.logspace(8, 10, num=20)

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model['coefficients']['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [50]:
num_non_zeros = 18
for l1_penalty2 in l1_penalty_values:
    modelPenRang = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=l1_penalty2, verbose = False)
    print 'number of nonzeros = %d' % (modelPenRang.coefficients['value']).nnz()
    print 'l1_penalty = %d' %(l1_penalty2)
    temp = num_non_zeros
    num_non_zeros = modelPenRang['coefficients']['value'].nnz()
    if num_non_zeros < temp:
        l1_penalty_min = l1_penalty2
    else :
        pass
    print 'l1_penalty_minima = %d' %(l1_penalty_min)
    #RSS.append(((validation['price'] - model_all.predict(validation))**2).sum())

number of nonzeros = 18
l1_penalty = 100000000
l1_penalty_minima = 7847599703
number of nonzeros = 18
l1_penalty = 127427498
l1_penalty_minima = 7847599703
number of nonzeros = 18
l1_penalty = 162377673
l1_penalty_minima = 7847599703
number of nonzeros = 18
l1_penalty = 206913808
l1_penalty_minima = 7847599703
number of nonzeros = 17
l1_penalty = 263665089
l1_penalty_minima = 263665089
number of nonzeros = 17
l1_penalty = 335981828
l1_penalty_minima = 263665089
number of nonzeros = 17
l1_penalty = 428133239
l1_penalty_minima = 263665089
number of nonzeros = 17
l1_penalty = 545559478
l1_penalty_minima = 263665089
number of nonzeros = 17
l1_penalty = 695192796
l1_penalty_minima = 263665089
number of nonzeros = 16
l1_penalty = 885866790
l1_penalty_minima = 885866790
number of nonzeros = 15
l1_penalty = 1128837891
l1_penalty_minima = 1128837891
number of nonzeros = 15
l1_penalty = 1438449888
l1_penalty_minima = 1128837891
number of nonzeros = 13
l1_penalty = 1832980710
l1_penalty_minima = 

In [ ]:
#number of nonzeros = 10
#l1_penalty = 2976351441
#l1_penalty_minima = 2976351441
#number of nonzeros = 6
#l1_penalty = 3792690190

In [35]:
print 'L1 min : ' + str(l1_penalty_min)

print 'L1 max : ' + str(l1_penalty_max)

L1 min : 10000000000.0
L1 max : 428133239.872


Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzero` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzero` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [55]:
model_selected = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty= 2976351441, verbose = False)

In [56]:
model_selected.coefficients.print_rows(18, 3)

+------------------+-------+---------------+
|       name       | index |     value     |
+------------------+-------+---------------+
|   (intercept)    |  None | 196100.937806 |
|     bedrooms     |  None | 2181.57432107 |
| bedrooms_square  |  None |      0.0      |
|    bathrooms     |  None | 17962.6966612 |
|   sqft_living    |  None | 34.1424656512 |
| sqft_living_sqrt |  None | 789.319789078 |
|     sqft_lot     |  None |      0.0      |
|  sqft_lot_sqrt   |  None |      0.0      |
|      floors      |  None |  3665.9308176 |
|  floors_square   |  None |      0.0      |
|    waterfront    |  None |      0.0      |
|       view       |  None | 11333.8410308 |
|    condition     |  None |      0.0      |
|      grade       |  None | 3578.90040044 |
|    sqft_above    |  None | 32.7432013718 |
|  sqft_basement   |  None | 12.7953811359 |
|     yr_built     |  None |      0.0      |
|   yr_renovated   |  None |      0.0      |
+------------------+-------+---------------+
[18 rows x

In [ ]:
l1_penalty_min = 
l1_penalty_max = 

***QUIZ QUESTIONS***

What values did you find for `l1_penalty_min` and`l1_penalty_max`? 

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [159]:
#number of nonzeros = 7
#l1_penalty limite inferior = 3413023616 confirmado 
#l1_penalty limite superior = 3614277247 confirmado



l1_penalty_min = 3413023615
l1_penalty_max = 3614277247
#3614277878

l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

for l1_penalty3 in l1_penalty_values:
    modelPenRang = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=l1_penalty3, verbose = False)
    print 'number of nonzeros = %d' % (modelPenRang.coefficients['value']).nnz()
    print 'l1_penalty = %d' %(l1_penalty3)
    


number of nonzeros = 8
l1_penalty = 3413023615
number of nonzeros = 7
l1_penalty = 3423615912
number of nonzeros = 7
l1_penalty = 3434208208
number of nonzeros = 7
l1_penalty = 3444800504
number of nonzeros = 7
l1_penalty = 3455392801
number of nonzeros = 7
l1_penalty = 3465985097
number of nonzeros = 7
l1_penalty = 3476577394
number of nonzeros = 7
l1_penalty = 3487169690
number of nonzeros = 7
l1_penalty = 3497761986
number of nonzeros = 7
l1_penalty = 3508354283
number of nonzeros = 7
l1_penalty = 3518946579
number of nonzeros = 7
l1_penalty = 3529538875
number of nonzeros = 7
l1_penalty = 3540131172
number of nonzeros = 7
l1_penalty = 3550723468
number of nonzeros = 7
l1_penalty = 3561315765
number of nonzeros = 7
l1_penalty = 3571908061
number of nonzeros = 7
l1_penalty = 3582500357
number of nonzeros = 7
l1_penalty = 3593092654
number of nonzeros = 7
l1_penalty = 3603684950
number of nonzeros = 7
l1_penalty = 3614277247


In [152]:
# calculo de RSS en Validation set:

RSS = []
#crss =[]
for l1_penalty in l1_penalty_values:
    model_all = graphlab.linear_regression.create(validation, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=l1_penalty, verbose = False)
    RSS.append(((validation['price'] - model_all.predict(validation))**2).sum())
    #crss.append([l1_penalty,((validation['price'] - model_all.predict(validation))**2).sum()]) # Para confirmar cuplas l1,rss




In [153]:
crss

[[3413023616.0, 1092600814292296.4],
 [3423615912.3684211, 1094082958492131.1],
 [3434208208.7368422, 1095568272419895.0],
 [3444800505.1052632, 1097056756075587.5],
 [3455392801.4736843, 1098548409459209.2],
 [3465985097.8421054, 1099945333928013.9],
 [3476577394.2105265, 1101032289091512.6],
 [3487169690.5789475, 1102120696692803.4],
 [3497761986.9473686, 1103210504122582.9],
 [3508354283.3157897, 1104301737668270.0],
 [3518946579.6842108, 1105418369013353.8],
 [3529538876.0526314, 1106599251575320.4],
 [3540131172.4210525, 1107781820168473.2],
 [3550723468.7894735, 1108968510271689.5],
 [3561315765.1578946, 1110158641264155.2],
 [3571908061.5263157, 1111350469370597.2],
 [3582500357.8947368, 1112543994591015.8],
 [3593092654.2631578, 1113739216925409.5],
 [3603684950.6315789, 1114936136373780.2],
 [3614277247.0, 1116134752936127.5]]

In [145]:
selected_l1_penalty  = l1_penalty_values[RSS.index(min(RSS))] 
print 'l1 seleccionada:   '+ str(selected_l1_penalty)
print 'valor min de RSS= %d' %(min(RSS))


l1 seleccionada:   3413023616.0
valor min de RSS= 1092600814292296


In [147]:
l1_penalty_values

array([  3.40600341e+09,   3.41696519e+09,   3.42792698e+09,
         3.43888876e+09,   3.44985054e+09,   3.46081232e+09,
         3.47177410e+09,   3.48273588e+09,   3.49369766e+09,
         3.50465944e+09,   3.51562122e+09,   3.52658300e+09,
         3.53754478e+09,   3.54850656e+09,   3.55946834e+09,
         3.57043012e+09,   3.58139190e+09,   3.59235369e+09,
         3.60331547e+09,   3.61427725e+09])

In [148]:
RSS

[1092600814292296.4,
 1094082958492131.1,
 1095568272419895.0,
 1097056756075587.5,
 1098548409459209.2,
 1099945333928013.9,
 1101032289091512.6,
 1102120696692803.4,
 1103210504122582.9,
 1104301737668270.0,
 1105418369013353.8,
 1106599251575320.4,
 1107781820168473.2,
 1108968510271689.5,
 1110158641264155.2,
 1111350469370597.2,
 1112543994591015.8,
 1113739216925409.5,
 1114936136373780.2,
 1116134752936127.5]

In [ ]:
# antes
#l1 seleccionada:   3406003414.0
#valor min de RSS= 1091620231990951

In [63]:
model_all.coefficients.print_rows(18, 3)

+------------------+-------+---------------+
|       name       | index |     value     |
+------------------+-------+---------------+
|   (intercept)    |  None | 234459.799722 |
|     bedrooms     |  None | 33.9774634621 |
| bedrooms_square  |  None |      0.0      |
|    bathrooms     |  None | 13507.9845961 |
|   sqft_living    |  None |  28.627476905 |
| sqft_living_sqrt |  None | 613.538510434 |
|     sqft_lot     |  None |      0.0      |
|  sqft_lot_sqrt   |  None |      0.0      |
|      floors      |  None |      0.0      |
|  floors_square   |  None |      0.0      |
|    waterfront    |  None |      0.0      |
|       view       |  None |      0.0      |
|    condition     |  None |      0.0      |
|      grade       |  None | 2643.19459258 |
|    sqft_above    |  None |  25.103285184 |
|  sqft_basement   |  None |      0.0      |
|     yr_built     |  None |      0.0      |
|   yr_renovated   |  None |      0.0      |
+------------------+-------+---------------+
[18 rows x

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzero`.

***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?